In [2]:
import fastbook
from fastbook import *
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from dtreeviz.trees import *
from IPython.display import Image, display_svg, SVG

pd.options.display.max_rows = 20
pd.options.display.max_columns = 8

In [3]:
import os
import pandas as pd
import numpy as np

cwd = os.getcwd()
print("Current working directory: {0}".format(cwd))

realestate = pd.read_csv("./Ames Real Estate Data.csv", low_memory = False)
HousePrice = pd.read_csv("./Ames_HousePrice.csv")

Current working directory: /home/koz/koz/NYCDSA/AmesWorking


In [4]:
realestate.shape

(22213, 91)

In [5]:
HousePrice.shape

(2580, 82)

In [7]:
HousePrice[:4]

,Unnamed: 0,PID,GrLivArea,SalePrice,...,MoSold,YrSold,SaleType,SaleCondition
0,1,909176150,856,126000,...,3,2010,WD,Normal
1,2,905476230,1049,139500,...,2,2009,WD,Normal
2,3,911128020,1001,124900,...,11,2007,WD,Normal
3,4,535377150,1039,114000,...,5,2009,WD,Normal


In [39]:
dep_var = 'SalePrice'
HousePrice[dep_var] = np.log(HousePrice[dep_var])

In [9]:
HousePrice[:4]["MSSubClass"]

0     30
1    120
2     30
3     70
Name: MSSubClass, dtype: int64

In [40]:
numeric_columns = HousePrice._get_numeric_data().columns
numeric_columns

Index(['Unnamed: 0', 'PID', 'GrLivArea', 'SalePrice', 'MSSubClass',
       'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt',
       'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
       'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'BsmtFullBath',
       'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
       'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea',
       'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold'],
      dtype='object')

In [41]:
numeric_df = HousePrice[list(set(numeric_columns) - set(['PID', 'Unnamed: 0',"MSSubClass"]))]

In [42]:
numeric_df.shape

(2580, 36)

In [36]:
missingRows = HousePrice.isnull().any(axis = 1)
missingCols = HousePrice.isnull().any(axis = 0)
HousePrice.columns[missingCols]

Index(['LotFrontage', 'Alley', 'MasVnrType', 'MasVnrArea', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Electrical',
       'BsmtFullBath', 'BsmtHalfBath', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PoolQC', 'Fence', 'MiscFeature'],
      dtype='object')

In [43]:
test_cond = numeric_df['YrSold'] == 2010
remainder_cond = numeric_df['YrSold'] < 2010
numeric_test = numeric_df[test_cond]
numeric_test.shape
numeric_train_and_valid = numeric_df[remainder_cond]
numeric_train_and_valid.shape

(2265, 36)

In [44]:
valid_cond = numeric_train_and_valid['YrSold'] == 2009
train_idx = np.where(~valid_cond)[0]
valid_idx = np.where(valid_cond)[0]

splits = (list(train_idx), list(valid_idx))

In [45]:
procs = [ Categorify, FillMissing ]

In [48]:
cont, cat = cont_cat_split(numeric_train_and_valid, 1, dep_var=dep_var)
# print(cont, cat)
# import pdb; pdb.set_trace()
tpo = TabularPandas(numeric_train_and_valid, procs, cont, cat, y_names=dep_var, splits=splits)
len(tpo.train), len(tpo.valid)

(1680, 585)

In [49]:
def root_mse(pred, y): return round(math.sqrt(((pred-y)**2).mean()), 6)
def model_rmse(m, xs, y): return root_mse( m.predict(xs), y)

In [50]:
xs, y = tpo.train.xs, tpo.train.y
valid_xs, valid_y = tpo.valid.xs, tpo.valid.y

In [51]:
m = DecisionTreeRegressor()
m.fit(xs, y)

DecisionTreeRegressor()

In [52]:
model_rmse(m, xs, y)

0.001154

In [53]:
model_rmse(m, valid_xs, valid_y)

0.33389

In [33]:
def rf(xs, y, n_estimators=400, max_samples=200,
       max_features=0.5, min_samples_leaf=5, **kwargs):
    return RandomForestRegressor(n_jobs=-1, n_estimators=n_estimators,
        max_samples=max_samples, max_features=max_features,
        min_samples_leaf=min_samples_leaf, oob_score=True).fit(xs, y)

In [54]:
m2 = rf(xs, y);

In [55]:
model_rmse(m2, xs, y)

0.134358

In [56]:
model_rmse(m2, valid_xs, valid_y)

0.257969